In [1]:
#libraries 
import numpy as np
import pandas as pd
import re
import os


#preprocessing
import string
from time import time  
from nltk.corpus import stopwords #remove stop words
chachedWords = stopwords.words('english')

#cosine similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import math
from collections import Counter


Base: https://medium.com/@adriensieg/text-similarities-da019229c894

REF: 
    
https://arxiv.org/pdf/1301.3781.pdf
    
https://www.aclweb.org/anthology/N19-1181.pdf
    
http://proceedings.mlr.press/v37/kusnerb15.pdf

https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37842.pdf

https://arxiv.org/pdf/1609.08144.pdf

In [2]:
#df = pd.read_csv(r"C:\Users\hirom\OneDrive - NOVAIMS\NOVA IMS\OneDrive_1_25-02-2021\Text Mining\Project\corpus\de-en\scores.csv")
df = pd.read_csv(r"C:\Users\hirom\Documents\GitHub\textmining_project\test\ru-en\scores.csv")
df

,source,reference,translation
0,Через полчаса обуглившийся клубень достают и п...,"After half an hour, the charred tuber is taken...","After half-an-hour, the charred tuber is retri..."
1,"Здесь никто не думает отменять смертную казнь,...","Here, no one thinks to abolish the death penal...","Here, no one is concerned with abolishing the ..."
2,"Собеседники ""Известий"" в ОНФ отмечают, что док...","The interlocutors of"" Izvestiya ""in the onf no...",Izvestia’s sources in the ONF note that the re...
3,На древней Венере могли существовать океаны.,On the ancient Venus could exist in the oceans.,Oceans could have existed on ancient Venus.
4,До этого момента убийства оставались лишь исто...,"Up to this point, the murders were just a stor...","Up until this point, the murders have remained..."
...,...,...,...
13152,"Председатель «BMW» заявил, что компания считае...",The chairman of BMW said the company believes ...,"The BMW chairman said the company believed ""th..."
13153,Из-за дисквалификации ВФЛА на Чемпионате мира ...,Because of the VFL disqualification at the Wor...,"On account of the ban against the ARAF, Russia..."
13154,Репетиции спектакля этого года начались в июне...,Rehearsals of this year's performance began in...,Rehearsals for this year's production began in...
13155,"Концерт пройдет в барнаульском клубе ""Фараон"" ...","The concert will be held at the club ""Pharaoh""...",The concert will be held at Barnaul’s “Pharaoh...


## Preprocessing 

In [9]:
df.isnull().sum()

source         0
reference      1
translation    0
dtype: int64

In [17]:
#filter the missing reference 
dfclean = df.dropna(axis=0)
dfclean.shape

(13156, 3)

In [3]:
def preprocessing2(sentence):
    return " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in sentence.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in chachedWords)
     

In [18]:
dfclean['p_reference'] = dfclean.apply(lambda x:preprocessing2(x['reference']),axis=1)

<ipython-input-18-90cc462c0668>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfclean['p_reference'] = dfclean.apply(lambda x:preprocessing2(x['reference']),axis=1)


In [19]:
dfclean['p_translation'] = dfclean.apply(lambda x:preprocessing2(x['translation']),axis=1)

<ipython-input-19-1e4834487e22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfclean['p_translation'] = dfclean.apply(lambda x:preprocessing2(x['translation']),axis=1)


In [20]:
def delete_sw(sentence):
    stop_words = stopwords.words('english')
    processed = sentence.split()
    clean = [w for w in processed if w not in chachedWords]
    return clean 

In [21]:
df_cp = dfclean.copy()

In [22]:
df_cp['w_reference'] = df_cp.apply(lambda x:delete_sw(x['p_reference']),axis=1)

In [23]:
df_cp['w_translation'] = df_cp.apply(lambda x:delete_sw(x['p_translation']),axis=1)

## Embedding

https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

Doc2Vec

In [24]:
# import
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(df_cp['w_translation'])]
tagged_data

C:\Users\hirom\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[TaggedDocument(words=['half', 'hour', 'charred', 'tuber', 'retrieved', 'devoured', 'eager', 'unwashed', 'hands'], tags=[0]),
 TaggedDocument(words=['one', 'concerned', 'abolishing', 'death', 'penalty', 'explains', 'lawyer', 'victor', 'lee', 'hong'], tags=[1]),
 TaggedDocument(words=['izvestia', 'sources', 'onf', 'note', 'report', 'head', 'state', 'consist', 'deep', 'analysis', 'situation', 'country', 'federal', 'districts'], tags=[2]),
 TaggedDocument(words=['oceans', 'could', 'existed', 'ancient', 'venus'], tags=[3]),
 TaggedDocument(words=['point', 'murders', 'remained', 'anecdote', 'standing', 'land', 'lynchings', 'took', 'place', 'air', 'feels', 'haunted'], tags=[4]),
 TaggedDocument(words=['generally', 'talking', 'private', 'events'], tags=[5]),
 TaggedDocument(words=['criminal', 'charges', 'filed', 'paragraph', 'part', 'article', 'criminal', 'code', 'russian', 'federation', 'hooliganism', 'part', 'article', 'paragraphs', 'f', 'part', 'article', 'criminal', 'code', 'russian', 'fe

In [25]:
## Train doc2vec model
model_Doc2Vec = Doc2Vec(tagged_data, vector_size = 1024, window = 3, min_count = 1.5, epochs = 100)



In [26]:
def Doc2Vec(sentece):
    x = model_Doc2Vec.infer_vector(sentece)    
    return x
    

In [ ]:
#embedding = df_cp.apply(lambda x:Doc2Vec(x['w_reference']), axis=1)

In [ ]:
#pd.DataFrame(embedding).to_csv('embedding.csv')

## Similarity 

## Doc2Vec + Cosine Similarity 

In [27]:
from scipy import spatial
def cosinesimilarity(ref,trans):
    t = model_Doc2Vec.infer_vector(trans)
    r = model_Doc2Vec.infer_vector(ref)
    result = 1 - spatial.distance.cosine(t,r)
    return result

REF: https://www.machinelearningplus.com/nlp/cosine-similarity/

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.

**Interpretation** : A cosine value of 0 means that the two vectors are at 90 degrees to each other (orthogonal) and have no match. The closer the cosine value to 1, the smaller the angle and the greater the match between vectors. 

In [28]:
df_cp['metric'] = df_cp.apply(lambda x:cosinesimilarity(x['w_reference'],x['w_translation']), axis=1)

In [29]:
df_cp

,source,reference,translation,p_reference,p_translation,w_reference,w_translation,metric
0,Через полчаса обуглившийся клубень достают и п...,"After half an hour, the charred tuber is taken...","After half-an-hour, the charred tuber is retri...",half hour charred tuber taken eaten impatient...,half an hour charred tuber retrieved devoured...,"[half, hour, charred, tuber, taken, eaten, imp...","[half, hour, charred, tuber, retrieved, devour...",0.388149
1,"Здесь никто не думает отменять смертную казнь,...","Here, no one thinks to abolish the death penal...","Here, no one is concerned with abolishing the ...",here one thinks abolish death penalty expl...,here one concerned abolishing death penalty ...,"[one, thinks, abolish, death, penalty, explain...","[one, concerned, abolishing, death, penalty, e...",0.755494
2,"Собеседники ""Известий"" в ОНФ отмечают, что док...","The interlocutors of"" Izvestiya ""in the onf no...",Izvestia’s sources in the ONF note that the re...,interlocutors of izvestiya in onf note repor...,izvestia s sources onf note report head state ...,"[interlocutors, izvestiya, onf, note, report, ...","[izvestia, sources, onf, note, report, head, s...",-0.017516
3,На древней Венере могли существовать океаны.,On the ancient Venus could exist in the oceans.,Oceans could have existed on ancient Venus.,ancient venus could exist oceans,oceans could existed ancient venus,"[ancient, venus, could, exist, oceans]","[oceans, could, existed, ancient, venus]",0.044763
4,До этого момента убийства оставались лишь исто...,"Up to this point, the murders were just a stor...","Up until this point, the murders have remained...",point murders story now place people lynche...,point murders remained anecdote now standin...,"[point, murders, story, place, people, lynched...","[point, murders, remained, anecdote, standing,...",-0.246172
...,...,...,...,...,...,...,...,...
13152,"Председатель «BMW» заявил, что компания считае...",The chairman of BMW said the company believes ...,"The BMW chairman said the company believed ""th...",chairman bmw said company believes there inte...,bmw chairman said company believed there inte...,"[chairman, bmw, said, company, believes, intel...","[bmw, chairman, said, company, believed, intel...",0.190347
13153,Из-за дисквалификации ВФЛА на Чемпионате мира ...,Because of the VFL disqualification at the Wor...,"On account of the ban against the ARAF, Russia...",vfl disqualification world championships londo...,account ban araf russian track field athletes...,"[vfl, disqualification, world, championships, ...","[account, ban, araf, russian, track, field, at...",0.407088
13154,Репетиции спектакля этого года начались в июне...,Rehearsals of this year's performance began in...,Rehearsals for this year's production began in...,rehearsals year s performance began june firs...,rehearsals year s production began june first...,"[rehearsals, year, performance, began, june, f...","[rehearsals, year, production, began, june, fi...",0.408637
13155,"Концерт пройдет в барнаульском клубе ""Фараон"" ...","The concert will be held at the club ""Pharaoh""...",The concert will be held at Barnaul’s “Pharaoh...,concert held club pharaoh october p m,concert held barnaul s pharaoh club october ...,"[concert, held, club, pharaoh, october, p]","[concert, held, barnaul, pharaoh, club, octobe...",0.491891


In [30]:
df_final = df_cp.drop(columns=['p_reference', 'p_translation','w_reference','w_translation'])

In [31]:
df_final

,source,reference,translation,metric
0,Через полчаса обуглившийся клубень достают и п...,"After half an hour, the charred tuber is taken...","After half-an-hour, the charred tuber is retri...",0.388149
1,"Здесь никто не думает отменять смертную казнь,...","Here, no one thinks to abolish the death penal...","Here, no one is concerned with abolishing the ...",0.755494
2,"Собеседники ""Известий"" в ОНФ отмечают, что док...","The interlocutors of"" Izvestiya ""in the onf no...",Izvestia’s sources in the ONF note that the re...,-0.017516
3,На древней Венере могли существовать океаны.,On the ancient Venus could exist in the oceans.,Oceans could have existed on ancient Venus.,0.044763
4,До этого момента убийства оставались лишь исто...,"Up to this point, the murders were just a stor...","Up until this point, the murders have remained...",-0.246172
...,...,...,...,...
13152,"Председатель «BMW» заявил, что компания считае...",The chairman of BMW said the company believes ...,"The BMW chairman said the company believed ""th...",0.190347
13153,Из-за дисквалификации ВФЛА на Чемпионате мира ...,Because of the VFL disqualification at the Wor...,"On account of the ban against the ARAF, Russia...",0.407088
13154,Репетиции спектакля этого года начались в июне...,Rehearsals of this year's performance began in...,Rehearsals for this year's production began in...,0.408637
13155,"Концерт пройдет в барнаульском клубе ""Фараон"" ...","The concert will be held at the club ""Pharaoh""...",The concert will be held at Barnaul’s “Pharaoh...,0.491891
